
### 1. **Contexto del Problema**

- **Marketplace con múltiples sellers:**  
  En el marketplace se pueden encontrar productos que, aunque son idénticos o muy similares, son vendidos por diferentes vendedores. Esto puede generar una experiencia confusa para el usuario, ya que se le presentan muchas opciones que en realidad corresponden al mismo producto o a variantes muy cercanas.

- **Accesibilidad de la información:**  
  La API de Mercado Libre permite obtener datos relevantes de cada producto, como los títulos y las imágenes, lo que posibilita analizar y comparar estos productos desde un punto de vista textual y visual.

---

### 2. **Objetivo Principal**

- **Identificar productos similares o idénticos:**  
  El reto consiste en desarrollar una estrategia que permita detectar, dentro de un gran volumen de ítems, aquellos que son iguales o muy parecidos, pese a provenir de distintos sellers.

- **Agrupar y hacer comparables estos ítems:**  
  Una vez identificados, el objetivo es agruparlos para que se puedan comparar de manera directa. Esto mejora la experiencia del usuario al facilitar la elección entre productos similares.

---

### 3. **Aspectos Técnicos y Metodológicos a Considerar**

- **Extracción de datos:**  
  - **Títulos:** Son fundamentales para el análisis textual y permiten evaluar similitudes en la descripción del producto.  
  - **Imágenes:** Pueden utilizarse para comparar visualmente los productos y confirmar si realmente se trata del mismo artículo o de uno muy similar.

- **Preprocesamiento y Transformación de Datos:**
  - **Texto:**  
    - Limpieza (minúsculas, eliminación de signos de puntuación, stop words, etc.).  
    - Tokenización y normalización (por ejemplo, lematización o stemming).  
    - Conversión a vectores mediante técnicas como TF-IDF o embeddings para capturar el significado de los títulos.
  - **Imágenes:**  
    - Descarga y procesamiento (redimensionamiento, normalización).  
    - Extracción de características visuales usando modelos pre-entrenados (por ejemplo, CNN como ResNet, VGG, etc.) para obtener embeddings de las imágenes.

- **Medición de similitud:**
  - **Entre textos:**  
    - Utilizar métricas como la similitud del coseno sobre los vectores de TF-IDF o embeddings para medir la proximidad semántica entre los títulos.
  - **Entre imágenes:**  
    - Calcular similitudes entre los embeddings de las imágenes, nuevamente empleando métricas como la similitud del coseno o distancia euclidiana.
  - **Fusión de Modalidades:**  
    - Combinar la similitud textual y la visual para obtener una puntuación global que determine qué tan similares son dos productos.

- **Agrupamiento (Clustering):**
  - Emplear técnicas de clustering (por ejemplo, DBSCAN, clustering jerárquico, K-Means, etc.) que permitan formar grupos de productos similares basándose en las puntuaciones de similitud.
  - Ajustar los parámetros del algoritmo para lograr clusters coherentes y minimizar falsos positivos o negativos.

---

### 4. **Impacto y Beneficios del Enfoque**

- **Mejora en la experiencia de usuario:**  
  Al agrupar ítems similares, el marketplace puede presentar opciones comparativas de manera más ordenada, evitando que el usuario se sienta abrumado por muchas opciones idénticas o muy parecidas.

- **Optimización en la toma de decisiones:**  
  Los compradores podrán evaluar mejor las opciones disponibles, considerando factores como precio, condiciones y reputación del seller, una vez que se han agrupado productos equivalentes.

- **Posibles extensiones:**  
  - Validar los grupos de productos mediante retroalimentación (feedback) de usuarios o expertos.
  - Implementar soluciones supervisadas si se dispone de datos etiquetados en el futuro, para afinar aún más la detección de productos similares.

---

### 5. **Resumen de lo que se Pide**

- **Problema:**  
  Detectar y agrupar productos que, pese a ser ofrecidos por distintos sellers, son idénticos o muy similares.

- **Datos a utilizar:**  
  Títulos y/o imágenes extraídas de la API de Mercado Libre (y, potencialmente, otros atributos relevantes).

- **Metodología a implementar:**  
  - Extracción y preprocesamiento de la información.  
  - Transformación de los datos en representaciones comparables (vectorización de texto y extracción de características de imágenes).  
  - Cálculo de similitudes entre productos.  
  - Aplicación de técnicas de clustering para formar grupos de ítems similares.

- **Objetivo final:**  
  Crear grupos de productos comparables que permitan al usuario diferenciar y elegir de forma más eficiente entre opciones muy similares, mejorando así la experiencia de compra en el marketplace.




## 1. **Exploración y Comprensión de la Estructura de Datos**

### a. Categorías y Atributos
- **Obtener el árbol de categorías:**  
  Utilizaremos el endpoint `/sites/MLA/categories` para recuperar todas las categorías del sitio (por ejemplo, Mercado Libre Argentina). Esto nos ayudará a entender en qué contextos se publican los ítems y a segmentarlos según su naturaleza.
  
- **Detalle de una categoría:**  
  Con endpoints como `/categories/{id}` (por ejemplo, `/categories/MLA1744` para “Autos y Camionetas”), podemos conocer la estructura jerárquica, los atributos obligatorios y opcionales, y la configuración específica de cada categoría. Esto es clave para saber qué campos (títulos, descripciones, imágenes, atributos técnicos, etc.) tenemos disponibles para comparar los productos.

- **Atributos específicos y valores top:**  
  Usaremos también `/categories/{id}/attributes` para conocer los atributos específicos de una categoría. Además, el endpoint de **top values** nos ayudará a identificar cuáles son los valores más comunes para ciertos atributos (por ejemplo, las marcas más usadas), lo que puede servir para validar o enriquecer nuestro análisis.

### b. Predictor de Categorías
- Con el recurso `/domain_discovery/search` podremos, a partir de un título de producto, predecir la categoría a la que pertenece. Esto es útil si queremos validar que estamos comparando ítems dentro de la misma categoría o si necesitamos filtrar productos que efectivamente son similares.

---

## 2. **Extracción de Datos: Cómo Recopilar la Información**

### a. Extracción Masiva mediante Búsqueda
- **Uso del endpoint de búsqueda:**  
  Emplearemos la llamada a `/sites/MLA/search` con parámetros `category`, `q`, `limit` y `offset` para extraer un conjunto representativo de publicaciones. Por ejemplo:
  ```bash
  curl -X GET "https://api.mercadolibre.com/sites/MLA/search?category=MLA1000&offset=0&limit=50"
  ```
  Esto nos permitirá recopilar títulos, imágenes y otros datos relevantes de cada ítem.

### b. Paginación y Manejo de Resultados
- **Limit y Offset:**  
  La documentación explica cómo utilizar los parámetros `limit` y `offset` para obtener bloques de resultados. Esto es esencial para recolectar una cantidad suficiente de datos sin sobrecargar el sistema y para realizar un muestreo representativo de los productos.

---

## 3. **Preprocesamiento y Enriquecimiento de la Información**

### a. Procesamiento de Texto (Títulos y Descripciones)
- **Limpieza y Vectorización:**  
  Una vez que extraigamos los títulos (y, si es posible, las descripciones) de los ítems, procederemos a limpiarlos (eliminando mayúsculas, signos de puntuación, stop words) y a transformarlos en vectores mediante técnicas como TF-IDF o embeddings (por ejemplo, usando Word2Vec o BERT).

### b. Procesamiento de Imágenes
- **Extracción de Features:**  
  Dado que la API nos permite descargar las imágenes, utilizaremos modelos pre-entrenados (por ejemplo, ResNet o VGG) para extraer embeddings de las imágenes. Estos embeddings se podrán comparar para medir la similitud visual entre productos.

---

## 4. **Cálculo de Similitud y Agrupación de Ítems**

### a. Cálculo de Similitud
- **Entre textos:**  
  Con los vectores obtenidos a partir de los títulos, aplicaremos la similitud del coseno para medir qué tan cercanos son dos ítems en función de su contenido textual.
- **Entre imágenes:**  
  De manera análoga, mediremos la similitud entre los embeddings de las imágenes.

### b. Fusión de Modalidades
- **Combinación de similitudes:**  
  Se diseñará un sistema de scoring que combine la similitud textual y la visual (posiblemente ponderando una sobre la otra según la calidad de los datos) para determinar una puntuación global de similitud.

### c. Clustering
- **Algoritmos de Clustering:**  
  Con la matriz de similitud, emplearemos algoritmos de clustering (como DBSCAN o clustering jerárquico) para agrupar los ítems que resulten muy similares o idénticos, identificando de esta manera productos ofrecidos por distintos sellers pero que corresponden al mismo artículo.

---

## 5. **Validación y Enriquecimiento del Análisis**

- **Validación Cruzada con Atributos:**  
  Aprovecharemos la información de atributos (por ejemplo, marca, modelo, año, etc.) extraída de los endpoints `/categories/{id}/attributes` para confirmar que los ítems agrupados son efectivamente comparables.
- **Uso del Predictor de Categorías:**  
  En casos donde el título sea poco claro, el recurso `/domain_discovery/search` nos ayudará a asignar la categoría correcta y, por lo tanto, a limitar el análisis a productos de una misma categoría.

---

## 6. **Implementación y Próximos Pasos**

- **Desarrollo en Python y Jupyter Notebooks:**  
  Se implementará el flujo de trabajo utilizando Python, donde se organizarán las llamadas a la API (usando la librería `requests`), el preprocesamiento de datos (con librerías como `pandas` y `scikit-learn`), y la visualización de resultados.
- **Repositorios y Documentación:**  
  Toda la solución se documentará en un repositorio público en GitHub, permitiendo la replicación y extensión del análisis.
- **Iteración y Mejora Continua:**  
  Se evaluará la calidad de los clusters generados y se realizarán ajustes en los parámetros de preprocesamiento, métodos de extracción de features y algoritmos de clustering, basándonos en pruebas y validaciones manuales.

---

## Resumen del Enfoque

1. **Explorar la API Docs** para conocer la estructura de categorías, atributos y métodos de búsqueda.
2. **Extraer datos** relevantes de los ítems (títulos, imágenes, atributos) utilizando los endpoints de categorías y búsqueda.
3. **Preprocesar y vectorizar** la información textual y visual.
4. **Calcular la similitud** entre ítems utilizando técnicas de NLP y procesamiento de imágenes.
5. **Agrupar los ítems** similares mediante algoritmos de clustering.
6. **Validar y ajustar** el modelo usando atributos específicos de cada categoría y el predictor de categorías.



In [2]:
import requests
import json

# Definimos la URL del endpoint de categorías para MLA
url_categorias = "https://api.mercadolibre.com/sites/MCO/categories"

# Realizamos la petición GET
respuesta = requests.get(url_categorias)

# Verificamos que la petición se haya realizado con éxito
if respuesta.status_code == 200:
    categorias = respuesta.json()
    # Imprimimos las primeras categorías para ver el formato
    print("Número de categorías obtenidas:", len(categorias))
    print(json.dumps(categorias[:5], indent=4, ensure_ascii=False))
else:
    print("Error en la solicitud:", respuesta.status_code)


Número de categorías obtenidas: 32
[
    {
        "id": "MCO1747",
        "name": "Accesorios para Vehículos"
    },
    {
        "id": "MCO441917",
        "name": "Agro"
    },
    {
        "id": "MCO1403",
        "name": "Alimentos y Bebidas"
    },
    {
        "id": "MCO1071",
        "name": "Animales y Mascotas"
    },
    {
        "id": "MCO1367",
        "name": "Antigüedades y Colecciones"
    }
]


In [3]:
# Definimos el ID de la categoría que queremos explorar
categoria_id = "MLA1744"
url_detalle_categoria = f"https://api.mercadolibre.com/categories/{categoria_id}"

respuesta_categoria = requests.get(url_detalle_categoria)

if respuesta_categoria.status_code == 200:
    detalle_categoria = respuesta_categoria.json()
    # Mostramos información relevante: nombre, atributos obligatorios, estructura jerárquica, etc.
    print("Detalle de la categoría:")
    print(json.dumps(detalle_categoria, indent=4, ensure_ascii=False))
else:
    print("Error en la solicitud de detalle de categoría:", respuesta_categoria.status_code)


Detalle de la categoría:
{
    "id": "MLA1744",
    "name": "Autos y Camionetas",
    "picture": "https://http2.mlstatic.com/storage/categories-api/images/470e5a62-5d07-432a-8f00-6a67b2ffada8.png",
    "permalink": null,
    "total_items_in_this_category": 102968,
    "path_from_root": [
        {
            "id": "MLA1743",
            "name": "Autos, Motos y Otros"
        },
        {
            "id": "MLA1744",
            "name": "Autos y Camionetas"
        }
    ],
    "children_categories": [],
    "attribute_types": "attributes",
    "settings": {
        "adult_content": false,
        "buying_allowed": false,
        "buying_modes": [
            "classified"
        ],
        "catalog_domain": "MLA-CARS_AND_VANS",
        "coverage_areas": "not_allowed",
        "currencies": [
            "USD",
            "ARS"
        ],
        "fragile": false,
        "immediate_payment": "optional",
        "item_conditions": [
            "used",
            "not_specified",
   